# Code zu der Bachelorarbeit:
# "Comparitve Study von Machine Learning Modellen zur Erkennung von Web Schwachstellen"
## von Nils Pudenz, 2735230

## Imports

In [24]:

#%pip install kaggle scikit-learn xgboost catboost tabpfn pandas numpy matplotlib seaborn -q
#%pip install --quiet scikit-learn xgboost catboost tabpfn chardet
#%pip install -U scikit-learn
## in deiner (Conda/venv) Umgebung
#%pip install --upgrade "torch==2.*" --index-url https://download.pytorch.org/whl/cu121
#%pip install --upgrade xgboost catboost scikit-learn pandas scipy tabpfn
##wenn es komplikationen mit torch gibt, deiinstallieren und neu installieren
#%pip uninstall torch
#%pip install torch --index-url https://download.pytorch.org/whl/cu121 --upgrade
#%pip install openpyxl


In [25]:
import os, math, time
import numpy as np
import pandas as pd
import torch

import zipfile
import random
from pathlib import Path
import matplotlib.pyplot as plt
import joblib
import sys

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from tabpfn import TabPFNClassifier

from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import make_pipeline


from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import resample
from kaggle.api.kaggle_api_extended import KaggleApi

from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support

from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV


## Check ob GPU verwendet werden kann

In [26]:
print(sys.version)
print("CUDA verfügbar:", torch.cuda.is_available())

3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
CUDA verfügbar: True


In [27]:
print("Python:", sys.executable)
print("Torch:", torch.__version__, "| CUDA build:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())

Python: c:\Users\nilsp\Github_Desktop\Comparative_Study_ML_WebVuln\.venv\Scripts\python.exe
Torch: 2.5.1+cu121 | CUDA build: 12.1
CUDA available: True


## mögliche Modeloptimization

In [28]:
'''
xgb = XGBClassifier(
    tree_method="hist",        # CPU-Optimierung
    n_estimators=400,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.9,
    colsample_bytree=0.8,
    n_jobs=-1,
    random_state=42
)

from catboost import CatBoostClassifier
cat = CatBoostClassifier(
    iterations=400,
    depth=8,
    learning_rate=0.1,
    random_seed=42,
    loss_function="Logloss",
    task_type="CPU",
    thread_count=8,
    od_type="Iter",            # early stopping
    od_wait=30,
    verbose=False
)
#'''

'\nxgb = XGBClassifier(\n    tree_method="hist",        # CPU-Optimierung\n    n_estimators=400,\n    max_depth=6,\n    learning_rate=0.1,\n    subsample=0.9,\n    colsample_bytree=0.8,\n    n_jobs=-1,\n    random_state=42\n)\n\nfrom catboost import CatBoostClassifier\ncat = CatBoostClassifier(\n    iterations=400,\n    depth=8,\n    learning_rate=0.1,\n    random_seed=42,\n    loss_function="Logloss",\n    task_type="CPU",\n    thread_count=8,\n    od_type="Iter",            # early stopping\n    od_wait=30,\n    verbose=False\n)\n#'

In [29]:
#Speicher optimieren
'''
vec = TfidfVectorizer(
    ngram_range=(3,5),
    max_features=50_000,
    sublinear_tf=True,
    lowercase=False
).fit(X_train_raw)           # nur einmal fitten
'''

'\nvec = TfidfVectorizer(\n    ngram_range=(3,5),\n    max_features=50_000,\n    sublinear_tf=True,\n    lowercase=False\n).fit(X_train_raw)           # nur einmal fitten\n'

In [30]:
#kleine Hyperparamsuche statt Grid-Overkill
'''
search_space = {
    "max_depth": [4, 6, 8],
    "learning_rate": [0.05, 0.1, 0.2],
    "n_estimators": [200, 400, 600]
}
randcv = RandomizedSearchCV(
    xgb,
    search_space,
    n_iter=10,            # statt 3×3×3 = 27
    scoring="f1",
    cv=3,
    n_jobs=-1
)
randcv.fit(X_train_vec, y_train)
'''

'\nsearch_space = {\n    "max_depth": [4, 6, 8],\n    "learning_rate": [0.05, 0.1, 0.2],\n    "n_estimators": [200, 400, 600]\n}\nrandcv = RandomizedSearchCV(\n    xgb,\n    search_space,\n    n_iter=10,            # statt 3×3×3 = 27\n    scoring="f1",\n    cv=3,\n    n_jobs=-1\n)\nrandcv.fit(X_train_vec, y_train)\n'

In [31]:
def predict_in_batches(model, X, batch_size=512): #um Überlastung zu vermeiden, evtl. 256 oder 128
    """Make predictions on input data in batches."""
    preds = []
    for i in range(0, X.shape[0], batch_size):
        batch = X[i:i + batch_size]
        preds.append(model.predict(batch))
    return np.concatenate(preds)


## Dowload Kaggle Datasets
Requires Kaggle API credentials ('~/.kaggle/kaggle.json') für API-Token, um zugriff auf die Datenbanken über das Kaggle Konto zu bekommen

In [32]:
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

In [33]:
#Dowload der Datasets von Kaggle, Output =1 erfoglgreich, Output = 0 fehlerhaft
os.system("kaggle datasets download -d syedsaqlainhussain/sql-injection-dataset -p data --unzip --quiet")
os.system("kaggle datasets download -d syedsaqlainhussain/cross-site-scripting-xss-dataset-for-deep-learning -p data --unzip --quiet")
#KAGGLE_DATASETS = { #gleich wie oben nur renaming auf sql & xss
#    "sql": "syedsaqlainhussain/sql-injection-dataset",
#    "xss": "syedsaqlainhussain/cross-site-scripting-xss-dataset-for-deep-learning"
#}

0

In [34]:
def kaggle_download(dataset, path="data", unzip=True):
    api = KaggleApi()
    api.authenticate() #nutzt ~/.kaggle/kaggle.json für Authentifizierung oder Environment-Variablen
    api.dataset_download_files(dataset, path=path, unzip=unzip)
    print(f"Downloaded {dataset}")

kaggle_download("syedsaqlainhussain/sql-injection-dataset")#, path=DATA_DIR, unzip=True)
kaggle_download("syedsaqlainhussain/cross-site-scripting-xss-dataset-for-deep-learning")#, path=DATA_DIR, unzip=True)
    

Dataset URL: https://www.kaggle.com/datasets/syedsaqlainhussain/sql-injection-dataset
Downloaded syedsaqlainhussain/sql-injection-dataset
Dataset URL: https://www.kaggle.com/datasets/syedsaqlainhussain/cross-site-scripting-xss-dataset-for-deep-learning
Downloaded syedsaqlainhussain/cross-site-scripting-xss-dataset-for-deep-learning


In [35]:

sql_df = pd.read_csv("data/SQLiV3.csv", encoding="utf-8", low_memory=False)
xss_df = pd.read_csv("data/XSS_dataset.csv", encoding="utf-8", low_memory=False)

In [36]:
# Spalten ansehen
print(sql_df.columns.tolist())

# Typische Index-/Hilfsspalten loswerden
sql_df = sql_df.loc[:, ~sql_df.columns.str.contains(r"^Unnamed|^index$", case=False)]

# Auf die Kernspalten reduzieren (falls etwas anderes drin ist)
sql_df = sql_df[["Sentence", "Label"]].copy()

# Optional: Duplikate auf Satzebene entfernen (falls noch nicht passiert)
sql_df = sql_df.drop_duplicates(subset=["Sentence"]).reset_index(drop=True)

print("clean shape:", sql_df.shape)  # Erwartung: (30873, 2)
print(sql_df["Label"].value_counts(normalize=True))


['Sentence', 'Label', 'Unnamed: 2', 'Unnamed: 3']
clean shape: (30873, 2)
Label
0                                                                         0.628891
1                                                                         0.370162
 --                                                                       0.000359
waitfor delay '0:0:__TIME__'--                                            0.000131
 DROP TABLE Suppliers                                                     0.000065
 desc users                                                               0.000033
SELECT *                                                                  0.000033
 OR                                                                       0.000033
 if not  (  select system_user  )   <> 'sa' waitfor delay '0:0:2' --      0.000033
 drop table temp --                                                       0.000033
 grant resource to name                                                   0.000033
 /*Sele

In [37]:

for name, df in {"SQL": sql_df, "XSS": xss_df}.items():
    print(f"{name} dataset shape: {df.shape}")
    display(df.head())
    display(df.describe())
    display(df.info())

SQL dataset shape: (30873, 2)


,Sentence,Label
0,""" or pg_sleep ( __TIME__ ) --",1
1,create user name identified by pass123 tempora...,NaN
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1


,Sentence,Label
count,30872,30619
unique,30872,17
top,""" or pg_sleep ( __TIME__ ) --",0
freq,1,19256


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30873 entries, 0 to 30872
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  30872 non-null  object
 1   Label     30619 non-null  object
dtypes: object(2)
memory usage: 482.5+ KB


None

XSS dataset shape: (13686, 3)


,Unnamed: 0,Sentence,Label
0,0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,2,"\t </span> <span class=""reference-text"">Steeri...",0
3,3,"\t </span> <span class=""reference-text""><cite ...",0
4,4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


,Unnamed: 0,Label
count,13686.000000,13686.000000
mean,6842.500000,0.538726
std,3950.952227,0.498516
min,0.000000,0.000000
25%,3421.250000,0.000000
50%,6842.500000,1.000000
75%,10263.750000,1.000000
max,13685.000000,1.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13686 entries, 0 to 13685
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  13686 non-null  int64 
 1   Sentence    13686 non-null  object
 2   Label       13686 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 320.9+ KB


None

## Basic Cleaning
* Drop Duplicate rows
* Handle missing values (simple fill-na)

In [38]:
for df in (sql_df, xss_df):
    df.drop_duplicates(inplace=True)
    df.fillna(0, inplace=True)

In [39]:
def preprocess_xy(df: pd.DataFrame,
                  label_candidates=("label", "class", "target"),
                  label_map=None):
    if label_map is None:
        label_map = {
            "0": "0", "1": "1",
            "benign": "0", "normal": "0", "legitimate": "0", "safe": "0",
            "attack": "1", "malicious": "1", "sql injection": "1",
            "sql-injection": "1", "xss": "1"
        }

    # Zielspalte finden (im *übergebenen* df!)
    cols_lower = {c.lower(): c for c in df.columns}
    target_col = next((cols_lower[c] for c in label_candidates if c in cols_lower), None)
    if target_col is None:
        raise ValueError(f"Keine Label-Spalte gefunden. Kandidaten: {label_candidates}")

    # Labels normieren -> nur 0/1 behalten
    y_str = df[target_col].astype(str).str.strip().str.lower()
    y_map = y_str.map(label_map)
    mask = y_map.notna()
    y = pd.to_numeric(y_map[mask]).astype(int).to_numpy()

    # Rohtext aus allen Nicht-Label-Spalten zusammenbauen
    feat_cols = [c for c in df.columns if c != target_col]
    X_raw = df.loc[mask, feat_cols].astype(str).agg(" ".join, axis=1)

    return X_raw, y, target_col


# Globale Settings

In [40]:

#Deterministische Ausgabe generieren, um die Reproduzierbarkeit zu gewährleisten
RANDOM_STATE = 42
#np.random.seed(RANDOM_STATE) wofür das?
#random.seed(RANDOM_STATE)
os.environ["OMP_NUM_THREADS"] = "8"
os.environ["OPENBLAS_NUM_THREADS"] = "8"
torch.set_num_threads(8)

USE_CUDA = torch.cuda.is_available()  # für TabPFN & XGBoost
print(f"[ENV] CUDA avail: {USE_CUDA} | Torch CUDA build: {torch.version.cuda}")


[ENV] CUDA avail: True | Torch CUDA build: 12.1


# Hilfsfunktionen 

## Evaluationsmetriken

In [41]:
def binary_metrics(y_true, y_pred) -> dict:
    """Präzision/Recall/F1 & Raten (FPR/FNR) für binäre Klassifikation."""
    y_pred = np.asarray(y_pred).ravel()
    p  = precision_score(y_true, y_pred)
    r  = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    fpr = float(fp / (fp + tn)) if (fp + tn) else 0.0
    fnr = float(fn / (fn + tp)) if (fn + tp) else 0.0
    return dict(Precision=p, Recall=r, F1=f1, FPR=fpr, FNR=fnr)

def evaluate_model(model, X_test, y_test, name, use_batches=False, batch_size=256) -> dict: #Dictionary für Evaluierungsmetriken
    """Zeitmessung + Vorhersage (optional in Batches) + Metriken."""
    print(f"→ Evaluate {name} on X_test={getattr(X_test,'shape',None)}")
    t0 = time.perf_counter()
    if use_batches:
        y_pred = predict_in_batches(model, X_test, batch_size=batch_size, verbose=True)
    else:
        y_pred = model.predict(X_test)
    pred_s = time.perf_counter() - t0
    m = binary_metrics(y_test, y_pred)
    res = dict(Model=name, Pred_s=pred_s, **m)
    return res

## Preprocessing

### Batchweise Vorhersage für TabPFN

In [42]:
def predict_in_batches(model, X, batch_size=256, verbose=False):
    """Vorhersage in Batches (schont RAM/VRAM; wichtig für TabPFN)."""
    n = X.shape[0]
    out = []
    total = math.ceil(n / batch_size)
    for b, i in enumerate(range(0, n, batch_size), start=1):
        j = min(i + batch_size, n)
        t1 = time.perf_counter()
        with torch.inference_mode():
            out.append(model.predict(X[i:j]))
        dt = time.perf_counter() - t1
        if verbose:
            print(f"   [predict] batch {b:>3}/{total} ({j-i} rows) in {dt:.2f}s")
        if USE_CUDA:
            torch.cuda.synchronize()
    return np.concatenate(out)

### Label bereinigung der Datensets

In [43]:
def clean_labels(df, label_col="Label", text_cols=("Sentence",)):
    """Bringt Labels robust auf {0,1} und gibt (X_raw, y) zurück."""
    df = df.copy()
    y_raw = df[label_col].astype(str).str.strip().str.lower()
    map01 = {"0": "0", "1": "1", "benign": "0", "normal": "0", "legitimate": "0", "safe": "0",
             "attack": "1", "malicious": "1", "sql injection": "1", "sql-injection": "1", "xss": "1"}
    y_map = y_raw.map(map01)
    mask = y_map.notna()
    y = pd.to_numeric(y_map[mask]).astype(int).to_numpy()
    feat_cols = [c for c in df.columns if c != label_col]
    X_raw = df.loc[mask, feat_cols].astype(str).agg(" ".join, axis=1)
    return X_raw, y

## Split

In [44]:
# Falls clean_labels(X) bereits (Rohtext, y) zurückgibt:
X_txt_sql, y_sql = clean_labels(sql_df, label_col="Label", text_cols=("Sentence",))
X_train_sql, X_test_sql, y_train_sql, y_test_sql = train_test_split(
    X_txt_sql, y_sql, test_size=0.2, stratify=y_sql, random_state=RANDOM_STATE
)

X_txt_xss, y_xss = clean_labels(xss_df, label_col="Label", text_cols=("Sentence",))
X_train_xss, X_test_xss, y_train_xss, y_test_xss = train_test_split(
    X_txt_xss, y_xss, test_size=0.2, stratify=y_xss, random_state=RANDOM_STATE
)

# Modelle

In [45]:
models = {
    "RandomForest": RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=RANDOM_STATE),
    "MLP": MLPClassifier(hidden_layer_sizes=(256,128), activation="relu",
                         early_stopping=True, n_iter_no_change=5, max_iter=200,
                         random_state=RANDOM_STATE),
    "XGBoost": XGBClassifier(
        n_estimators=500, max_depth=6, learning_rate=0.1,
        subsample=0.9, colsample_bytree=0.8,
        tree_method="hist", device=("cuda" if USE_CUDA else "cpu"),
        random_state=RANDOM_STATE
    ),
    "CatBoost": CatBoostClassifier(
        iterations=400, depth=8, learning_rate=0.1,
        loss_function="Logloss", random_seed=RANDOM_STATE, verbose=False,
        task_type="CPU"   # stabil über Pool + Sparse; bei GPU: task_type="GPU" und ggf. Dense verwenden
    ),
    "TabPFN": TabPFNClassifier(
        device=("cuda" if USE_CUDA else "cpu"),
        ignore_pretraining_limits=True
    )
}

# Pipeline

In [46]:
#TDF-IDF Einstellungen
VEC_ARGS = dict(ngram_range=(3,5), max_features=50_000, sublinear_tf=True, lowercase=False)
#VEC_ARGS = dict(analyzer="char", ngram_range=(3,5), min_df=3, max_features=50_000, sublinear_tf=True, lowercase=False, dtype=np.float32)


TABPFN_MAX_SAMPLES = 4000     # starte konservativ; später 6000/8000 testen
TABPFN_N_COMPONENTS = 150     # <=500, kleiner = schneller/ram-sparender
TABPFN_BATCH = 128            # Batch für Predict; 128/64 bei RAM-Engpässen

results = []



## Pipeline je Datensatz (TF-IDF; TabPFN: SVD + batched predict)

In [47]:
# ===== vorausgesetzt: deine fertigen Splits existieren =====
# X_train_sql, X_test_sql, y_train_sql, y_test_sql
# X_train_xss, X_test_xss, y_train_xss, y_test_xss

from sklearn.base import clone
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from catboost import Pool
import numpy as np, time, torch


splits = {
    "SQL": (X_train_sql, X_test_sql, y_train_sql, y_test_sql),
    "XSS": (X_train_xss, X_test_xss, y_train_xss, y_test_xss),
}

for ds_name, (X_train_txt, X_test_txt, y_train, y_test) in splits.items():
    print("\n" + "="*70)
    print(f"DATASET: {ds_name} | train={len(y_train)} test={len(y_test)} (pos_rate_train={np.mean(y_train):.3f})")

    #TF-IDF nur auf Train fitten (kein Leakage)
    vec = TfidfVectorizer(**VEC_ARGS)
    t0 = time.perf_counter()
    X_train_vec = vec.fit_transform(X_train_txt)
    X_test_vec  = vec.transform(X_test_txt)
    print(f"[VEC] TF-IDF train={X_train_vec.shape}, test={X_test_vec.shape} in {time.perf_counter()-t0:.2f}s")

    #Modelle trainieren/evaluieren (jeweils frischen Klon verwenden)
    for name, base_model in models.items():
        model = clone(base_model)
        print("-"*50 + f"\nMODEL: {name} on {ds_name}")

        if name == "CatBoost":
            # CatBoost: Pool nutzen (sparse ok)
            pool_train = Pool(X_train_vec, y_train)
            t0 = time.perf_counter()
            model.fit(pool_train)
            print(f"[{name}] fit in {time.perf_counter()-t0:.2f}s")

            # Test direkt auf X_test_vec (CatBoost kann CSR); alternativ: Pool(X_test_vec, y_test)
            res = evaluate_model(model, X_test_vec, y_test, f"{name}-{ds_name}")
            results.append(res)
            print(res)

        elif name == "TabPFN":
            # TabPFN: ggf. stratifiziertes Subset (Limits), danach SVD -> dichte float32
            X_tab, y_tab = X_train_vec, y_train
            if X_train_vec.shape[0] > TABPFN_MAX_SAMPLES:
                sss = StratifiedShuffleSplit(n_splits=1, train_size=TABPFN_MAX_SAMPLES, random_state=RANDOM_STATE)
                idx, _ = next(sss.split(np.zeros(len(y_train)), y_train))
                X_tab = X_train_vec[idx]
                y_tab = np.asarray(y_train)[idx]
            print(f"[{name}] train subset: {X_tab.shape[0]} rows")

            t0 = time.perf_counter()
            svd = TruncatedSVD(n_components=TABPFN_N_COMPONENTS, random_state=RANDOM_STATE)
            X_train_svd = svd.fit_transform(X_tab).astype("float32", copy=False)
            X_test_svd  = svd.transform(X_test_vec).astype("float32", copy=False)
            print(f"[{name}] SVD train={X_train_svd.shape}, test={X_test_svd.shape} in {time.perf_counter()-t0:.2f}s")

            # Gerät einstellen & fitten
            if hasattr(model, "set_params"):
                model.set_params(device=("cuda" if USE_CUDA else "cpu"), ignore_pretraining_limits=True)
            t0 = time.perf_counter()
            model.fit(X_train_svd, y_tab)
            if USE_CUDA:
                torch.cuda.synchronize()
            print(f"[{name}] fit in {time.perf_counter()-t0:.2f}s (device={'cuda' if USE_CUDA else 'cpu'})")

            # Evaluieren (wenn deine evaluate_model batched predict unterstützt, ansonsten normal)
            try:
                res = evaluate_model(model, X_test_svd, y_test, f"{name}-{ds_name}",
                                     use_batches=True, batch_size=TABPFN_BATCH)
            except TypeError:
                res = evaluate_model(model, X_test_svd, y_test, f"{name}-{ds_name}")
            results.append(res)
            print(res)

        else:
            # RF / MLP / XGBoost: direkt auf sparse TF-IDF
            t0 = time.perf_counter()
            model.fit(X_train_vec, y_train)
            print(f"[{name}] fit in {time.perf_counter()-t0:.2f}s")
            res = evaluate_model(model, X_test_vec, y_test, f"{name}-{ds_name}")
            results.append(res)
            print(res)

# Danach kannst du z. B.:
# df_results = pd.DataFrame(results)
# display(df_results)
# df_results.to_excel("reports/results_from_presplit.xlsx", index=False)


DATASET: SQL | train=24675 test=6169 (pos_rate_train=0.367)
[VEC] TF-IDF train=(24675, 50000), test=(6169, 50000) in 1.11s
--------------------------------------------------
MODEL: RandomForest on SQL
[RandomForest] fit in 14.05s
→ Evaluate RandomForest-SQL on X_test=(6169, 50000)
{'Model': 'RandomForest-SQL', 'Pred_s': 0.8627100996673107, 'Precision': 0.9917172832689122, 'Recall': 0.792236435818262, 'F1': 0.8808239333006376, 'FPR': 0.0038441824705279346, 'FNR': 0.20776356418173797}
--------------------------------------------------
MODEL: MLP on SQL
[MLP] fit in 812.74s
→ Evaluate MLP-SQL on X_test=(6169, 50000)
{'Model': 'MLP-SQL', 'Pred_s': 0.02129790000617504, 'Precision': 0.9911894273127754, 'Recall': 0.7940008822232024, 'F1': 0.881704628949302, 'FPR': 0.004100461301896463, 'FNR': 0.20599911777679752}
--------------------------------------------------
MODEL: XGBoost on SQL
[XGBoost] fit in 10.33s
→ Evaluate XGBoost-SQL on X_test=(6169, 50000)
{'Model': 'XGBoost-SQL', 'Pred_s': 0.

c:\Users\nilsp\Github_Desktop\Comparative_Study_ML_WebVuln\.venv\Lib\site-packages\xgboost\core.py:729: UserWarning: [17:36:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


[CatBoost] fit in 136.01s
→ Evaluate CatBoost-SQL on X_test=(6169, 50000)
{'Model': 'CatBoost-SQL', 'Pred_s': 0.13463389966636896, 'Precision': 1.0, 'Recall': 0.7878253198059109, 'F1': 0.8813224771773994, 'FPR': 0.0, 'FNR': 0.2121746801940891}
--------------------------------------------------
MODEL: TabPFN on SQL
[TabPFN] train subset: 4000 rows
[TabPFN] SVD train=(4000, 150), test=(6169, 150) in 3.05s
[TabPFN] fit in 68.69s (device=cuda)
→ Evaluate TabPFN-SQL on X_test=(6169, 150)
   [predict] batch   1/49 (128 rows) in 27.49s
   [predict] batch   2/49 (128 rows) in 33.68s
   [predict] batch   3/49 (128 rows) in 51.46s
   [predict] batch   4/49 (128 rows) in 51.10s
   [predict] batch   5/49 (128 rows) in 52.48s
   [predict] batch   6/49 (128 rows) in 51.37s
   [predict] batch   7/49 (128 rows) in 49.35s
   [predict] batch   8/49 (128 rows) in 698.13s
   [predict] batch   9/49 (128 rows) in 262.15s
   [predict] batch  10/49 (128 rows) in 48.11s
   [predict] batch  11/49 (128 rows) in 

# Ergebnisse für Modelle

## Ergebnistabelle ausgeben

In [48]:
# Ergebnis-Tabelle ausgeben
df_results = pd.DataFrame(results)
display(df_results.sort_values(["Model"]).reset_index(drop=True))
df_results.to_csv("results_all.csv", index=False)
with pd.ExcelWriter("results_all.xlsx") as w:
    df_results.to_excel(w, sheet_name="All", index=False)
print(" Ergebnisse gespeichert: results_all.csv / results_all.xlsx")

,Model,Pred_s,Precision,Recall,F1,FPR,FNR
0,CatBoost-SQL,0.134634,1.000000,0.787825,0.881322,0.000000,0.212175
1,CatBoost-XSS,0.077010,1.000000,0.946441,0.972483,0.000000,0.053559
2,MLP-SQL,0.021298,0.991189,0.794001,0.881705,0.004100,0.205999
3,MLP-XSS,0.010082,0.999288,0.951864,0.975000,0.000792,0.048136
4,RandomForest-SQL,0.862710,0.991717,0.792236,0.880824,0.003844,0.207764
5,RandomForest-XSS,0.090351,1.000000,0.945085,0.971767,0.000000,0.054915
6,TabPFN-SQL,5056.038700,1.000000,0.783855,0.878833,0.000000,0.216145
7,TabPFN-XSS,909.746705,1.000000,0.942373,0.970332,0.000000,0.057627
8,XGBoost-SQL,0.041133,1.000000,0.423908,0.595415,0.000000,0.576092
9,XGBoost-XSS,0.023783,0.646363,1.000000,0.785201,0.638955,0.000000


 Ergebnisse gespeichert: results_all.csv / results_all.xlsx


## Ergebnisse speichern

## Ergebnistabelle

In [49]:
results_df = pd.DataFrame(results)
results_df.sort_values(["Model"], inplace=True)
print("\n Gesamt Ergebnisse: \n", results_df)

results_df.to_csv("results.csv", index=False)


 Gesamt Ergebnisse: 
               Model       Pred_s  Precision    Recall        F1       FPR  \
3      CatBoost-SQL     0.134634   1.000000  0.787825  0.881322  0.000000   
8      CatBoost-XSS     0.077010   1.000000  0.946441  0.972483  0.000000   
1           MLP-SQL     0.021298   0.991189  0.794001  0.881705  0.004100   
6           MLP-XSS     0.010082   0.999288  0.951864  0.975000  0.000792   
0  RandomForest-SQL     0.862710   0.991717  0.792236  0.880824  0.003844   
5  RandomForest-XSS     0.090351   1.000000  0.945085  0.971767  0.000000   
4        TabPFN-SQL  5056.038700   1.000000  0.783855  0.878833  0.000000   
9        TabPFN-XSS   909.746705   1.000000  0.942373  0.970332  0.000000   
2       XGBoost-SQL     0.041133   1.000000  0.423908  0.595415  0.000000   
7       XGBoost-XSS     0.023783   0.646363  1.000000  0.785201  0.638955   

        FNR  
3  0.212175  
8  0.053559  
1  0.205999  
6  0.048136  
0  0.207764  
5  0.054915  
4  0.216145  
9  0.057627  
2  

# Hyperparameter-Tuning & Cross-Validation

In [52]:
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import (
    HalvingRandomSearchCV, train_test_split, StratifiedKFold, StratifiedShuffleSplit, RandomizedSearchCV)
from joblib import Memory

## Feature und Modelle

In [53]:
# Feature-Builder (schnell & schlank)

def make_tfidf(max_features=20_000):
    return TfidfVectorizer(
        analyzer="char", ngram_range=(3,5),
        min_df=3, max_features=max_features,
        sublinear_tf=True, lowercase=False,
        dtype=np.float32
    )

# Cache, damit TF-IDF/SVD nicht in jedem Kandidaten neu gelernt werden müssen
memory = Memory(location="skcache", verbose=0)


#schlanke Suchräume
def pos_weight(y):
    pos = max(1, int(np.sum(y))); neg = max(1, int(len(y)-pos))
    return neg / pos

def grids(y):
    return {
        "RF": {
            "svd__n_components": [120, 150, 200],
            "clf__n_estimators": [300, 600],
            "clf__max_depth": [None, 20, 40],
            "clf__min_samples_split": [2, 5, 10],
            "clf__max_features": ["sqrt", None],
        },
        "MLP": {
            "svd__n_components": [120, 150, 200],
            "clf__hidden_layer_sizes": [(256,128), (512,256)],
            "clf__alpha": np.logspace(-5, -3, 3),
            "clf__learning_rate_init": [1e-4, 5e-4, 1e-3],
            "clf__batch_size": [64, 128],
        },
        "XGB": {
            "tfidf__max_features": [15_000, 20_000],
            "clf__n_estimators": [300, 500],
            "clf__max_depth": [4, 6, 8],
            "clf__learning_rate": [0.05, 0.1],
            "clf__subsample": [0.7, 1.0],
            "clf__colsample_bytree": [0.6, 1.0],
            "clf__reg_lambda": [0, 5],
            "clf__gamma": [0, 1],
            "clf__scale_pos_weight": [pos_weight(y)],
        },
        "CAT": {
            "tfidf__max_features": [15_000, 20_000],
            "clf__iterations": [400, 800],
            "clf__depth": [6, 8],
            "clf__learning_rate": [0.05, 0.1],
            "clf__l2_leaf_reg": [3, 5],
        }
    }



## Pipeline

In [ ]:
#Pipelines
def pipe_rf_svd(max_features=20_000, n_comp=150):
    return Pipeline([
        ("tfidf", make_tfidf(max_features)),
        ("svd", TruncatedSVD(n_components=n_comp, random_state=RANDOM_STATE)),
        ("clf", RandomForestClassifier(n_jobs=-1, random_state=RANDOM_STATE))
    ], memory=memory)

def pipe_mlp_svd(max_features=20_000, n_comp=150):
    return Pipeline([
        ("tfidf", make_tfidf(max_features)),
        ("svd", TruncatedSVD(n_components=n_comp, random_state=RANDOM_STATE)),
        ("scaler", StandardScaler(with_mean=True)),
        ("clf", MLPClassifier(
            early_stopping=True, n_iter_no_change=8, max_iter=200,
            random_state=RANDOM_STATE
        ))
    ], memory=memory)

def pipe_xgb_sparse(max_features=20_000):
    return Pipeline([
        ("tfidf", make_tfidf(max_features)),
        ("clf", XGBClassifier(
            tree_method="hist",
            device=("cuda" if USE_CUDA else "cpu"),
            n_estimators=400,
            random_state=RANDOM_STATE,
            n_jobs=1  # Parallelisierung über CV, nicht im Estimator
        ))
    ], memory=memory)

def pipe_cat_sparse(max_features=20_000):
    return Pipeline([
        ("tfidf", make_tfidf(max_features)),
        ("clf", CatBoostClassifier(
            loss_function="Logloss",
            verbose=False,
            random_seed=RANDOM_STATE
        ))
    ], memory=memory)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, f1_score

# Gemeinsames CV-Objekt für faire Vergleiche
cv_shared = StratifiedKFold(n_splits=2, shuffle=True, random_state=RANDOM_STATE)

# Hilfsfunktion für Hyperparameter-Suche
def make_search(estimator, param_distributions, y_train, use_halving=True, cv=None):
    if cv is None:
        cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=RANDOM_STATE)
    scorer = make_scorer(f1_score)
    if use_halving:
        n = len(y_train)
        min_res = max(200, int(0.15 * n))
        return HalvingRandomSearchCV(
            estimator=estimator,
            param_distributions=param_distributions,
            resource="n_samples",
            min_resources=min_res,
            max_resources="auto",
            factor=3,
            scoring=scorer,
            cv=cv,
            random_state=RANDOM_STATE,
            n_jobs=2,
            verbose=2
        )
    else:
        return RandomizedSearchCV(
            estimator=estimator,
            param_distributions=param_distributions,
            n_iter=10,
            scoring=scorer,
            cv=cv,
            random_state=RANDOM_STATE,
            n_jobs=2,
            verbose=2
        )

# Training + Suche + Refit + Threshold
def tune_one(name, X_train_txt, y_train, cv):
    if name == "RF":   pipe = pipe_rf_svd()
    elif name == "MLP": pipe = pipe_mlp_svd()
    elif name == "XGB": pipe = pipe_xgb_sparse()
    elif name == "CAT": pipe = pipe_cat_sparse()
    else: raise ValueError(name)

    # Teilmenge für HPO
    X_hpo, y_hpo = get_hpo_subset(X_train_txt, y_train, n=8000)
    space = grids(y_hpo)[name]
    search = make_search(pipe, space, y_hpo, use_halving=True, cv=cv)

    t0 = time.perf_counter()
    search.fit(X_hpo, y_hpo)
    dt = time.perf_counter() - t0
    print(f"[{name}] best F1 (CV): {search.best_score_:.4f} in {dt/60:.1f} min")
    print(f"[{name}] best params:", search.best_params_)

    best = search.best_estimator_.set_params(**search.best_params_)
    best.fit(X_train_txt, y_train)
    return best, search.best_params_, search.best_score_, dt

# Threshold-Optimierung + Testauswertung
def eval_on_test(name, est, X_train_txt, y_train, X_test_txt, y_test, tune_threshold=True):
    thr = 0.5
    if tune_threshold and hasattr(est, "predict_proba"):
        X_tr2, X_val, y_tr2, y_val = train_test_split(X_train_txt, y_train, test_size=0.15, stratify=y_train, random_state=RANDOM_STATE)
        est_for_thr = est
        est_for_thr.fit(X_tr2, y_tr2)
        p = est_for_thr.predict_proba(X_val)[:, 1]
        grid = np.linspace(0.1, 0.9, 81)
        thr = float(grid[int(np.argmax([f1_score(y_val, (p >= t).astype(int)) for t in grid]))])

    y_pred = (est.predict_proba(X_test_txt)[:,1] >= thr).astype(int) if hasattr(est, "predict_proba") else est.predict(X_test_txt)
    p, r, f1, _ = precision_recall_fscore_support(y_test, y_pred, average="binary", zero_division=0)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    fpr = float(fp / (fp + tn)) if (fp + tn) else 0.0
    fnr = float(fn / (fn + tp)) if (fn + tp) else 0.0
    return dict(Model=name, Thr=thr, Precision=p, Recall=r, F1=f1, FPR=fpr, FNR=fnr)

# Ausführung mit übergebenem Split
def run_fast_from_split(X_train_txt, X_test_txt, y_train, y_test, ds_name, cv):
    results = []
    for short, label in [("RF", "RandomForest"), ("MLP", "MLP"), ("XGB", "XGBoost"), ("CAT", "CatBoost")]:
        print("\n" + "=" * 60)
        print(f"TUNING: {label} ({ds_name})")
        best_est, params, best_cv, dt = tune_one(short, X_train_txt, y_train, cv=cv)
        res = eval_on_test(f"{label}-{ds_name}", best_est, X_train_txt, y_train, X_test_txt, y_test)
        print("TEST:", res)
        results.append(res)
    return pd.DataFrame(results)

# Anwendung auf beide Datensätze
sql_res = run_fast_from_split(X_train_sql, X_test_sql, y_train_sql, y_test_sql, ds_name="SQL", cv=cv_shared)
xss_res = run_fast_from_split(X_train_xss, X_test_xss, y_train_xss, y_test_xss, ds_name="XSS", cv=cv_shared)

display(sql_res)
display(xss_res)

In [ ]:
# ---------- AUSFÜHREN & Excel speichern ----------
from datetime import datetime

all_test, all_cv = [], []
for ds_name, df in [("SQL", sql_df), ("XSS", xss_df)]:
    df_test, df_cv = run_all_for_dataset(df, ds_name)
    all_test.append(df_test); all_cv.append(df_cv)

df_test_all = pd.concat(all_test, ignore_index=True)
df_cv_all   = pd.concat(all_cv, ignore_index=True)

display(df_test_all); display(df_cv_all)

# Excel-Export
outdir = Path("reports"); outdir.mkdir(parents=True, exist_ok=True)
ts = datetime.now().strftime("%Y%m%d-%H%M%S")
xlsx_path = outdir / f"results_hpo_{ts}.xlsx"
with pd.ExcelWriter(xlsx_path, engine="xlsxwriter") as w:
    df_test_all.to_excel(w, sheet_name="Test", index=False)
    df_cv_all.to_excel(w, sheet_name="CV",   index=False)
print(" Saved:", xlsx_path)